#### Create Datasets by randomly mixing 10min segments of preictal and interictal data.

In [ ]:
#convert .mat data into ndarray and compress it usingle pickle
def mat_to_numpy(files):
    print('Converting files ' + files[0] + ' to ' + files[-1])
    data = list()
    for idx, fl in enumerate(files):
        #open mat file
        mat = loadmat(fl)
        #convert from ndarray object to ndarray
        names = mat['dataStruct'].dtype.names
        ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
        #detect 100% drop-out, all data is zero or std too low
        if np.count_nonzero(ndata['data']) < 10:
            print('WARNING: 100% drop-out in file', fl)
            continue
        sigma = np.std(ndata['data'])
        if sigma < 1.0:
            print('WARNING: Low std =', sigma, 'in file', fl)
            continue
        #detect if columns out of order
        if not all(x<y for x, y in zip(ndata['channelIndices'][0], ndata['channelIndices'][0][1:])):
            print('WARNING: Columns out of order in file' + fl)
        # Normalize data: axis = 0 is to normalize along columns, vertical lines.
        #todo: Is it ok to normalize or not?
        data.append((ndata['data'] - np.mean(ndata['data'], axis=0))/sigma)
    #convert list to ndarray
    return np.asarray(data)

nrOfFiles = 20

def dataset(folder, name_pickle):
    #get files
    files_preictal = glob.glob(folder + "/*1.mat")
    files_interictal = glob.glob(folder + "/*0.mat")
    #randomize read files
    np.random.shuffle(files_preictal)
    np.random.shuffle(files_interictal)
    #read files into ndarrays
    data_preictal = mat_to_numpy(files_preictal[:nrOfFiles])
    labels_preictal = np.ones(data_preictal.shape[0], dtype=np.int32)
    data_interictal = mat_to_numpy(files_interictal[:nrOfFiles])
    labels_interictal = np.zeros(data_interictal.shape[0], dtype=np.int32)
    #concatenate preictal and interictal data
    print('Merging and shuffling data..')
    data_merge = np.concatenate((data_preictal, data_interictal), axis=0)
    labels_merge = np.concatenate((labels_preictal, labels_interictal), axis=0)
    #shuffle data and labels mantaining relation between them
    shuffle_idx = np.random.permutation(data_merge.shape[0])
    data_merge = data_merge[shuffle_idx,:,:]
    labels_merge = labels_merge[shuffle_idx]
    #bundle data and labels in dictionary
    print(data_merge.shape)
    save = {'data': data_merge, 'labels': labels_merge}
    try:
        with open(name_pickle, 'wb') as f:
            print('Pickling to ' + name_pickle)
            pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to' + name_pickle + ' :', e)
    
dataset('./data/train_1', './data/trainsh1_norm.pickle')

print('dataset done')